::: {.callout-note collapse="true"}
## Learning Outcomes
* Recognize the value of feature engineering as a tool to improve model performance
* Implement polynominal feature generation and one hot encoding
* Understand the interactions between model complexity, model variance, and training error
:::

At this point in the course, we've equipped ourselves with some powerful techniques to build and optimize models. We've explored how to develop models of multiple variables, as well as how to fit these models to maximize their performance.

All of this was done with one major caveat: the regression models we've worked with so far are all **linear in the input variables**. We've assumed that our predictions should be some combination of linear variables. While this works well in some cases, the real world isn't always so straightforward. In today's lecture, we'll learn an important method to address this issue – and consider some new problems that can arise when we do so.

## Feature Functions

Let's take a moment to remind ourselves where this linearity comes from. Consider the following dataset on vehicles:

In [ ]:
import seaborn as sns
vehicles = sns.load_dataset("mpg").rename(columns={"horsepower":"hp"}).dropna()
vehicles.head(5)

Suppose we wish to develop a model to predict a vehicle's fuel efficiency (`"mpg"`) as a function of its horsepower (`"hp"`). Glancing at the plot below, we see that the relationship between `"mpg"` and `"hp"` is non-linear – an SLR fit doesn't capture the relationship between the two variables.

<img src="images/degree_1.png" alt='degree_1' width='600'>

Recall our standard multiple linear regression model. In its current form, it is linear in terms of both $\theta_i$ and $x$:

$$\hat{y} = \theta_0 + \theta_1 x + \theta_2 x\:+\:...$$

Just by eyeballing the `vehicle` data plotted above, it seems that a *quadratic* model might be more appropriate. In other words, a model of the form below would likely do a better job of capturing the non-linear relationship between the two variables:

$$\hat{y} = \theta_0 + \theta_1 x + \theta_2 x^2$$

This looks fairly similar to our original multiple regression framework! Importantly, it is **still linear in $\theta_i$** – the prediction $\hat{y}$ is a linear combination of the model parameters. This means that we can use the same linear algebra methods as before to derive the optimal model parameters when fitting the model.

You may be wondering: how can this be a linear model if there is now a $x^2$ term? Although the model contains non-linear $x$ terms, it is linear with respect to the *model parameters*, $\theta_i$. Because our OLS derivation relied on assuming a linear model of $\theta_i$, the method is still valid to fit this new model.

If we refit the model with `"hp"` squared as its own feature, we see that the model follows the data much more closely.

$$\hat{\text{mpg}} = \theta_0 + \theta_1 (\text{hp}) + \theta_2 (\text{hp})^2$$

<img src="images/degree_2.png" alt='degree_2' width='600'>

Looks much better! What we've done here is called **feature engineering**: the process of transforming the raw features of a dataset into more informative features for modeling. By squaring the `"hp"` feature, we were able to create a new feature that signficantly improved the quality of our model.

We perform feature engineering by defining a **feature function**. A feature function is some function applied to the original variables in the data to generate one or more new features. More formally, a feature function is said to take a $d$ dimensional input and transform it to a $p$ dimensional input. This results in a new, feature-engineered design matrix that we rename $\Phi$.

$$\mathbb{X} \in \mathbb{R}^{n \times d} \longrightarrow \Phi \in \mathbb{R}^{n \times p}$$

In the `vehicles` example above, we applied a feature function to transform the original input with $d=1$ features into an engineered design matrix with $p=2$ features.

## One Hot Encoding
Feature engineering opens up a whole new set of possibilities for designing better performing models. As you will see in lab and homework, feature engineering is one of the most important parts of the entire modeling process.

A particularly powerful use of feature engineering is to allow us to perform regression on non-numeric features. **One hot encoding** is a feature engineering technique that generates numeric features from categorical data, allowing us to use our usual methods to fit a regression model on the data. 

To illustrate how this works, we'll refer back to the `tips` data from last lecture. Consider the `"day"` column of the dataset:

In [ ]:
import numpy as np
np.random.seed(1337)
tips = sns.load_dataset("tips").sample(100)
tips[["day"]].head(5)

   At first glance, it doesn't seem possible to fit a regression model to this data – we can't directly perform any mathematical operations on the entry "Thur". 
  
To resolve this, we instead create a new table with a feature for each unique value in the original `"day"` column. We then iterate through the `"day"` column. For each entry in `"day"` we fill the corresponding feature in the new table with 1. All other features are set to 0.

<img src="images/ohe.png" alt='ohe' width='600'>

This can be implemented in code using `pd.get_dummies` to generate the one hot encoding, then calling `pd.concat` to combine these new features with the original DataFrame.

In [ ]:
#| code-fold: false
import pandas as pd

# Perform the one hot encoding
ohe = pd.get_dummies(tips["day"])

# Combine with original features
tips = pd.concat([tips, ohe], axis=1)
tips.head(5)

Now, the data is ready to fit a model.

## Variance and Training Error

We've seen now that feature engineering allows us to build all sorts of features to improve the performance of the model. In particular, we saw that designing a more complex feature (squaring `"hp"` in the `vehicles` data previously) substantially improved the model's ability to capture non-linear relationships. To take full advantage of this, we might be inclined to design increasingly complex features. Consider the following three models, each of different order (the maximum exponent power of each model):

* Model with order 1: $\hat{\text{mpg}} = \theta_0 + \theta_1 (\text{hp})$
* Model with order 2: $\hat{\text{mpg}} = \theta_0 + \theta_1 (\text{hp}) + \theta_2 (\text{hp})^2$
* Model with order 4: $\hat{\text{mpg}} = \theta_0 + \theta_1 (\text{hp}) + \theta_2 (\text{hp})^2 + \theta_3 (\text{hp})^3 + \theta_4 (\text{hp})^4$

<br/>

<img src="images/degree_comparison.png" alt='degree_comparison' width='800'>

When we use our model to make predictions on the same data that was used to fit the model, we find that the MSE decreases with increasingly complex models. The **training error** is the model's error when generating predictions from the same data that was used for training purposes. We can conclude that the training error goes down as the complexity of the model increases. 

<img src="images/train_error.png" alt='train_error' width='500'>

This seems like good news – when working on the **training data**, we can improve model performance by designing increasingly complex models. 

However, high model complexity comes with its own set of issues. When a model has many complicated features, it becomes increasingly sensitive to the data used to fit it. Even a small variation in the data points used to train the model may result in wildly different results for the fitted model. The plots below illustrate this idea. In each case, we've fit a model to two very similar sets of data (in fact, they only differ by two data points!). Notice that the model with order 2 appears roughly the same across the two sets of data; in contrast, the model with order 4 changes erratically across the two datasets.

<img src="images/model_variance.png" alt='model_variance' width='600'>

The sensitivity of the model to the data used to train it is called the **model variance**. As we saw above, model variance tends to increase with model complexity. 

<img src="images/bvt.png" alt='bvt' width='500'>

## Overfitting

We can see that there is a clear "trade-off" that comes from the complexity of our model. As model complexity increases, the model's error on the training data decreases. At the same time, the model's variance tends to increase.

Why does this matter? To answer this question, let's take a moment to review our modeling workflow when making predictions on new data. 

1. Sample a dataset of training data from the real world
2. Use this training data to fit a model
3. Apply this fitted model to generate predictions on unseen data 

This first step – sampling training data – is important to remember in our analysis. As we saw above, a highly complex model may produce results that vary wildly across different samples of training data. If we happen to sample a set of training data that is a poor representation of the population we are trying to model, our model may perform poorly on any new set of data it has not seen before.

To see why, consider a model fit using the training data shown on the left. Because the model is so complex, it achieves zero error on the training set – it perfectly predicts each value in the training data! When we go to use this model to make predictions on a new sample of data, however, things aren't so good. The model now has enormous error on the unseen data. 

<img src="images/overfit.png" alt='overfit' width='600'>

The phenomenon above is called **overfitting**. The model effectively just memorized the training data it encountered when it was fitted, leaving it unable to handle new situations. 

The takeaway here: we need to strike a balance in the complexity of our models. A model that is too simple won't be able to capture the key relationships between our variables of interest; a model that is too complex runs the risk of overfitting. 

This begs the question: how do we control the complexity of a model? Stay tuned for the next lecture.
